In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_table('benchmark.txt', header = None)

In [44]:
df.head(20)

,0
0,start
1,benchmark start : 2020/12/24 12:12:48
2,Number of GPUs on current device : 1
3,CUDA Version : 11.0
4,Cudnn Version : 8005
5,Device Name : GeForce RTX 3090
6,"uname_result(system='Linux', node='gyh-X11DPi-..."
7,"scpufreq(current=1182.0009166666669, min=1000...."
8,cpu_count: 36
9,memory_available: 129118310400


In [159]:
# 使用正则提取数据
pattern1 = 'Benchmarking (\w+) (\w+) precision type (\w+)'
pattern2 = '(\w+)  model average (\w+) time :  (.+) ms'
bench_1 = df[0].str.extract(pattern1).rename(columns={0:'status_x',1:'precision',2:'model_x'}).dropna().reset_index(drop = True)
bench_2 = df[0].str.extract(pattern2).rename(columns={0:'model_y',1:'status_y',2:'time'}).dropna().reset_index(drop = True)

In [176]:
# 拼接两个dataframe，删除重复列及修改列名
bench = pd.concat([bench_1,bench_2], axis=1).drop(['model_y','status_y'],axis=1).rename(columns = {'status_x':'status','model_x':'model'})

In [177]:
# 修改时间精度
bench['time'] = bench['time'].astype('float').round(decimals = 3)

In [178]:
# 修改数据格式
bench.loc[bench['status']=='Training','status'] = 'Train'

In [179]:
# 长表转宽表
bench = bench.pivot(index='model',
                    columns=['status','precision'],values='time')
bench

status,Train,Inference,Train,Inference,Train,Inference
precision,float,float,half,half,double,double
model,,,,,,
densenet121,93.357,15.637,88.976,19.772,417.207,144.111
densenet161,136.624,31.750,144.319,27.555,1290.287,511.177
densenet169,104.840,21.598,121.556,26.371,511.404,175.808
densenet201,129.334,26.169,118.940,33.394,654.365,223.960
mnasnet0_5,28.528,8.039,27.198,6.929,48.232,11.870
mnasnet0_75,34.105,6.246,28.787,6.010,70.395,18.154
mnasnet1_0,34.314,6.553,29.600,6.785,88.913,25.796
mnasnet1_3,35.557,6.871,35.175,6.457,118.835,35.082


In [180]:
# 压缩多级列名
new_columns = bench.columns.map(lambda x :(x[0]+'_'+x[1]))
bench.columns = new_columns
bench.head()

,Train_float,Inference_float,Train_half,Inference_half,Train_double,Inference_double
model,,,,,,
densenet121,93.357,15.637,88.976,19.772,417.207,144.111
densenet161,136.624,31.750,144.319,27.555,1290.287,511.177
densenet169,104.840,21.598,121.556,26.371,511.404,175.808
densenet201,129.334,26.169,118.940,33.394,654.365,223.960
mnasnet0_5,28.528,8.039,27.198,6.929,48.232,11.870


In [181]:
# 调整数据
bench.index.name = None
bench = bench[['Train_half','Train_float','Train_double','Inference_half','Inference_float','Inference_double']]
bench.head()

,Train_half,Train_float,Train_double,Inference_half,Inference_float,Inference_double
densenet121,88.976,93.357,417.207,19.772,15.637,144.111
densenet161,144.319,136.624,1290.287,27.555,31.750,511.177
densenet169,121.556,104.840,511.404,26.371,21.598,175.808
densenet201,118.940,129.334,654.365,33.394,26.169,223.960
mnasnet0_5,27.198,28.528,48.232,6.929,8.039,11.870
